In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = pd.read_csv("output_file.csv")

# Drop unnecessary columns
data.drop(['Date', 'Time'], axis=1, inplace=True)

# Separate features and target variable
X = data.drop('Radiation', axis=1)
y = data['Radiation']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [2]:
from sklearn.neural_network import MLPRegressor

def create_ann():
    # Define the neural network structure
    ann = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=200, random_state=42)
    return ann

In [3]:
import numpy as np
from tqdm import tqdm
from sklearn.exceptions import ConvergenceWarning
import warnings

# Suppress convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Define WDO parameters
num_particles = 30
max_iterations = 100
c1 = 0.1  # Pressure Gradient force
c2 = 0.1  # Coriolis constant
c3 = 0.1  # Friction constant
c4 = 0.1  # Gravitational constant
V_max = 1  # Maximum velocity

# Initialize particles and velocities
particles = np.random.uniform(-1, 1, (num_particles, X_train.shape[1]))
velocities = np.random.uniform(-1, 1, (num_particles, X_train.shape[1]))

# Define the fitness function
def fitness_function(particle):
    ann = create_ann()
    ann.fit(X_train, y_train)
    predictions = ann.predict(X_test)
    mse = np.mean((predictions - y_test) ** 2)
    return mse

# Main WDO loop with progress bar
for iteration in tqdm(range(max_iterations), desc="WDO Training"):
    fitness_values = []
    for i in range(num_particles):
        fitness = fitness_function(particles[i])
        fitness_values.append(fitness)
        
        # Update velocities
        velocities[i] = velocities[i] + c1 * np.random.rand() * (np.mean(particles, axis=0) - particles[i])
        velocities[i] = velocities[i] - c2 * np.random.rand() * velocities[i]
        velocities[i] = velocities[i] - c3 * np.sign(velocities[i]) * (velocities[i] ** 2)
        velocities[i] = velocities[i] + c4 * np.random.rand() * (particles[i] - np.mean(particles, axis=0))
        
        # Ensure velocity is within bounds
        velocities[i] = np.clip(velocities[i], -V_max, V_max)
        
        # Update positions
        particles[i] = particles[i] + velocities[i]

# Select the best particle
best_particle_idx = np.argmin(fitness_values)
best_particle = particles[best_particle_idx]

# Train the final ANN with the best parameters
final_ann = create_ann()
final_ann.fit(X_train, y_train)

# Evaluate the final model
final_predictions = final_ann.predict(X_test)
final_mse = np.mean((final_predictions - y_test) ** 2)
print("Final MSE:", final_mse)

WDO Training: 100%|██████████| 100/100 [01:56<00:00,  1.16s/it]

Final MSE: 105519.41592342166
